In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [2]:
daily_data = pd.read_csv("/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv",index_col = 0)
submission = pd.read_csv("/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv",index_col = 0)

In [3]:
from sklearn.preprocessing import MinMaxScaler
feaures_to_scale =['temperature_celsius','wind_kph','pressure_mb','precip_mm','humidity','cloud','feels_like_celsius','visibility_km','uv_index','gust_kph']
scaler = MinMaxScaler()
daily_data[feaures_to_scale] = scaler.fit_transform(daily_data[feaures_to_scale])

In [4]:
X_test = daily_data[daily_data['condition_text'].isna()]
X_test['city_id'] = X_test['city_id'].astype(str)
X_test['city_id'] = X_test['city_id'].str[1:].astype(int)

X = daily_data[~(daily_data['condition_text'].isna())]
X['city_id'] = X['city_id'].astype(str)
X['city_id'] = X['city_id'].str[1:].astype(int)


print(X_test)
print(X)

        city_id  temperature_celsius condition_text  wind_kph  wind_degree  \
day_id                                                                       
D0001         1             0.617021            NaN  0.035411          210   
D0002         1             0.510638            NaN  0.035411          170   
D0005         1             0.425532            NaN  0.000000           92   
D0006         1             0.468085            NaN  0.000000           96   
D0008         1             0.489362            NaN  0.235127          330   
...         ...                  ...            ...       ...          ...   
D2888       112             0.457447            NaN  0.184136          113   
D2890       112             0.412766            NaN  0.133144           51   
D2891       112             0.451064            NaN  0.111898           46   
D2892       112             0.451064            NaN  0.152975           76   
D2893       112             0.438298            NaN  0.194051   

/tmp/ipykernel_18/1778906801.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['city_id'] = X_test['city_id'].astype(str)
/tmp/ipykernel_18/1778906801.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['city_id'] = X_test['city_id'].str[1:].astype(int)
/tmp/ipykernel_18/1778906801.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [5]:
y = X['condition_text']

In [6]:
X_test.drop(['sunrise','sunset','condition_text','feels_like_celsius','air_quality_us-epa-index','uv_index'],axis=1,inplace=True)
X.drop(['sunrise','sunset','condition_text','feels_like_celsius','air_quality_us-epa-index','uv_index'],axis=1,inplace=True)

/tmp/ipykernel_18/2861738084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(['sunrise','sunset','condition_text','feels_like_celsius','air_quality_us-epa-index','uv_index'],axis=1,inplace=True)
/tmp/ipykernel_18/2861738084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(['sunrise','sunset','condition_text','feels_like_celsius','air_quality_us-epa-index','uv_index'],axis=1,inplace=True)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
model =RandomForestClassifier(
    n_estimators=4200,
    criterion='entropy',
    max_depth=20,
    #min_samples_split=2,
    #min_samples_leaf=1,
    #min_weight_fraction_leaf=0.005,
    #max_features=None,
    #max_leaf_nodes=65,
    #min_impurity_decrease=0.0005,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=42,
    verbose=0,
    warm_start=False,
    #class_weight='balanced',
    ccp_alpha=0.075,
    max_samples=None
)
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')  # 5-fold cross-validation
print("Cross-validation accuracy scores:", scores)
print("Mean accuracy:", np.mean(scores))
print("Standard deviation of accuracy:", np.std(scores))
model.fit(X, y)
y_pred = model.predict(X)
print('train accuracy' + str(accuracy_score(y_pred,y)))


Cross-validation accuracy scores: [0.625      0.64583333 0.65625    0.71875    0.63157895]
Mean accuracy: 0.6554824561403508
Standard deviation of accuracy: 0.033457539253543916
train accuracy0.7014613778705637


1500

0.6868201754385964  
/0.7055482456140351   
/0.7117763157894738

/0.7139035087719299

In [10]:
y_pred = model.predict(X)
print(accuracy_score(y_pred,y))

0.7014613778705637


In [11]:
y_pred = model.predict(X_test)
daily_data.loc[daily_data['condition_text'].isna(), 'condition_text'] = y_pred
submission['condition_text']= daily_data['condition_text']
print(submission)
submission.to_csv('submission.csv')

                 condition_text
day_id                         
D0001             Partly Cloudy
D0002             Partly Cloudy
D0003   Light Rain with Thunder
D0004           Clear and Sunny
D0005           Clear and Sunny
...                         ...
D2889           Clear and Sunny
D2890           Clear and Sunny
D2891           Clear and Sunny
D2892           Clear and Sunny
D2893           Clear and Sunny

[2893 rows x 1 columns]
